# 워드투벡터까지 함수화하기
- 소수점 4자리까지만 나오게 하는 것도 함

In [1]:
import requests
import pandas as pd
from dotenv import load_dotenv
load_dotenv('../05_data_scraping/.env')
import os
# from sqlalchemy import create_engine
# import pymysql
# pymysql.install_as_MySQLdb()
# from dbio import mood_to_db, db_connect
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
import koreanize_matplotlib
import time
from konlpy.tag import Mecab
mecab = Mecab()
# from nltk.probability import FreqDist
# from wordcloud import WordCloud
import re
from datetime import datetime
from gensim.models import Word2Vec

/home/user/miniforge3/envs/project/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [27]:
def string_delete(text) :
    return text.replace("<b>","").replace("</b>", "").replace("&quot;", "")

def parse_date(date_str):
    # 날짜 문자열을 datetime 객체로 변환
    return datetime.strptime(date_str, "%Y%m%d")

def make_df(df_name) :
    for i in df_name :
        for idx in range(0, len(df_name[i])) : 
            df_name[i][idx] = string_delete(df_name[i][idx])
    df_name['postdate'] = df_name['postdate'].apply(parse_date)
    return df_name
    
def w2v(keyword) :
    client_id = os.getenv('client_id') # 환경 변수에 접근
    client_pw = os.getenv('client_pw') # 환경 변수에 접근

    stopwords = pd.read_csv('https://raw.githubusercontent.com/haram4th/ablearn/main/%ED%95%9C%EA%B5%AD%EC%96%B4%EB%B6%88%EC%9A%A9%EC%96%B4100.txt', header = None)
    ex_stop_words = ['사람', '집', '시간']
    for i in ex_stop_words : 
        stopwords = stopwords.drop(stopwords[stopwords[0] == i].index)
    stopwords.reset_index(drop=True, inplace=True)
    
    url = "https://openapi.naver.com/v1/search/blog.json"
    final_data_keyword = pd.DataFrame()
    start = time.time() # 시작
    for i in range(1, 10) :
        payload = {'query' : keyword, 'display' : 100, 'start' : i, 'sort' : 'sim'}
        headers = {'X-Naver-Client-Id' : client_id, 'X-Naver-Client-Secret' : client_pw}
        r = requests.get(url, params = payload, headers = headers)
        if r.status_code == 200 :
            data = r.json()
        else :
            print("Error Code:", r.status_code)
            
        df_keyword_sim = pd.json_normalize(data['items'])
        entire_data_keyword_sim = make_df(df_keyword_sim)
        final_data_keyword = pd.concat([final_data_keyword, entire_data_keyword_sim], ignore_index=True)
    # print(final_data_keyword)
    final_data_keyword['title2'] = final_data_keyword['title'].apply(lambda x: mecab.pos(re.sub(r"[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", x)))

    target = 'NNG', 'NNP', 'MAG', 'XR', 'NR', 'VV', 'VA', 'VCP', 'VCN', 'MM', 'IC', 'XPN', 'UNKNOWN'
    def extract_words(words_list) : 
        return [word for word, pos in words_list if pos in target]
    final_data_keyword['filtered'] = final_data_keyword['title2'].apply(lambda x : extract_words(x))
    final_data_keyword['filtered_list'] = final_data_keyword['filtered'].apply(lambda x : eval(x) if isinstance(x, str) else x)
    
    model_keyword_pos = Word2Vec(final_data_keyword['filtered_list'], vector_size=100, window=5, min_count=5, workers=8, sg=0)
    # 단어 50개 뽑기
    model_keyword_pos_result = model_keyword_pos.wv.most_similar(keyword, topn = 50)
    # 워드투벡터 결과 소수점 4자리로 변경하기
    w2v_result = list((item[0], round(item[1], 4)) for item in model_keyword_pos_result)
    print(f"{time.time()-start:.4f} sec")
    return keyword, w2v_result

In [28]:
result = w2v('홍제동')
result

1.4305 sec


('홍제동',
 [('맛집', 0.9986),
  ('홍제', 0.9985),
  ('찾', 0.9985),
  ('서대문구', 0.9984),
  ('평', 0.9984),
  ('곳', 0.9983),
  ('실화', 0.9981),
  ('하', 0.9981),
  ('치과', 0.998),
  ('이', 0.9978),
  ('화재', 0.9978),
  ('니스', 0.9977),
  ('서대문', 0.9977),
  ('잘', 0.9977),
  ('아파트', 0.9976),
  ('정보', 0.9976),
  ('근처', 0.9975),
  ('차', 0.9975),
  ('점', 0.9974),
  ('업체', 0.9973),
  ('일', 0.9973),
  ('먹', 0.9972),
  ('장', 0.9972),
  ('집', 0.9971),
  ('사건', 0.9971),
  ('원룸', 0.9971),
  ('포장', 0.9971),
  ('홍', 0.997),
  ('주차', 0.997),
  ('주택', 0.997),
  ('국수', 0.997),
  ('후라이드', 0.9969),
  ('카페', 0.9968),
  ('젓갈', 0.9968),
  ('호텔', 0.9968),
  ('헬스', 0.9968),
  ('더', 0.9967),
  ('김밥', 0.9967),
  ('병원', 0.9967),
  ('안', 0.9967),
  ('소방관', 0.9967),
  ('추천', 0.9967),
  ('플란트', 0.9967),
  ('챌린지', 0.9966),
  ('치료', 0.9965),
  ('신상', 0.9965),
  ('의원', 0.9964),
  ('양념', 0.9964),
  ('강릉', 0.9964),
  ('원', 0.9964)])